In [1]:
#ch20_SparkML_01.ipynb

In [2]:
#Step1.2 讀取文字檔，並查看資料筆數,
row_df=sqlContext.read.format("csv").option("header","true").option("delimiter","\t").load("train.tsv")

In [3]:
type(row_df) #pyspark.sql.dataframe.DataFrame

pyspark.sql.dataframe.DataFrame

In [4]:
row_df.count() #row_df 筆數

7395

In [5]:
#spark2.0  以 SparkSession 取代 sqlContext
#from pyspark.sql import SparkSession

In [6]:
#spark = SparkSession \
#    .builder \
#    .appName("Python Spark SQL basic example") \
#    .config("spark.some.config.option", "some-value") \
#    .getOrCreate()

In [7]:
#df = spark.read.csv("train.tsv")

In [8]:
#df.show()

In [9]:
#Step1.3 查看 schema, DataFrame.printSchema()
row_df.printSchema()

root
 |-- url: string (nullable = true)
 |-- urlid: string (nullable = true)
 |-- boilerplate: string (nullable = true)
 |-- alchemy_category: string (nullable = true)
 |-- alchemy_category_score: string (nullable = true)
 |-- avglinksize: string (nullable = true)
 |-- commonlinkratio_1: string (nullable = true)
 |-- commonlinkratio_2: string (nullable = true)
 |-- commonlinkratio_3: string (nullable = true)
 |-- commonlinkratio_4: string (nullable = true)
 |-- compression_ratio: string (nullable = true)
 |-- embed_ratio: string (nullable = true)
 |-- framebased: string (nullable = true)
 |-- frameTagRatio: string (nullable = true)
 |-- hasDomainLink: string (nullable = true)
 |-- html_ratio: string (nullable = true)
 |-- image_ratio: string (nullable = true)
 |-- is_news: string (nullable = true)
 |-- lengthyLinkDomain: string (nullable = true)
 |-- linkwordscore: string (nullable = true)
 |-- news_front_page: string (nullable = true)
 |-- non_markup_alphanum_characters: string (nulla

In [10]:
#Spep1.4 查看前10筆資料,並以select要檢視的欄位,DataFrame.select() method
row_df.select('url','alchemy_category','alchemy_category_score','is_news','label').show(10)

+--------------------+------------------+----------------------+-------+-----+
|                 url|  alchemy_category|alchemy_category_score|is_news|label|
+--------------------+------------------+----------------------+-------+-----+
|http://www.bloomb...|          business|              0.789131|      1|    0|
|http://www.popsci...|        recreation|              0.574147|      1|    1|
|http://www.menshe...|            health|              0.996526|      1|    1|
|http://www.dumbli...|            health|              0.801248|      1|    1|
|http://bleacherre...|            sports|              0.719157|      1|    0|
|http://www.conven...|                 ?|                     ?|      ?|    0|
|http://gofashionl...|arts_entertainment|               0.22111|      1|    1|
|http://www.inside...|                 ?|                     ?|      ?|    0|
|http://www.valetm...|                 ?|                     ?|      1|    1|
|http://www.howswe...|                 ?|           

In [11]:
#2.處理缺失值”?”, 使用DataFrame 的UDF (User Define Function)
#Step2.1 建立 DataFrame UDF 
from pyspark.sql.functions import udf #import udf 

In [12]:
def replace_question(x):         #define 一個 python function 'replace_qusetion()
    return("0" if x=="?" else x) 

In [13]:
replace_question=udf(replace_question)  #以udf() method 將python function 轉成 UDF

In [14]:
#Step2.2 import col module, 以 col module 讀取欄位資料. import types module 後續作資料型別轉換時使用
from pyspark.sql.functions import col
import pyspark.sql.types

In [15]:
#Step2.3 將DataFrame row_df 中第4到最後欄位 ([4:]) 轉換成double,除最後一個欄位是label其餘欄位是feature
#              並以UDF replace_question 處理（替代）缺失值 "?"

In [16]:
df=row_df.select(                          #DataFrame.select([]) , 參數是list型態  
    ['url','alchemy_category']+       #'url','alchemy_category' 這兩個欄位不轉換
    [replace_question(col(c)).cast("double").alias(c)
    for c in row_df.columns[4:]])   #for c in row_df.columns[4:]  --> 從 row_df 第4個到最後欄位 逐一取出
                                                       #col(c) --> 以col() 取值, replace_question(col(c)) --> 如果值是 "?" 以UDF replace_question 取代為 "0"
                                                       #.cast("double") 強至型別轉換, .alias(c) 以原欄位名稱作別名

In [17]:
#Step2.4 檢視整形後的DataFrame 'df'
df.printSchema()

root
 |-- url: string (nullable = true)
 |-- alchemy_category: string (nullable = true)
 |-- alchemy_category_score: double (nullable = true)
 |-- avglinksize: double (nullable = true)
 |-- commonlinkratio_1: double (nullable = true)
 |-- commonlinkratio_2: double (nullable = true)
 |-- commonlinkratio_3: double (nullable = true)
 |-- commonlinkratio_4: double (nullable = true)
 |-- compression_ratio: double (nullable = true)
 |-- embed_ratio: double (nullable = true)
 |-- framebased: double (nullable = true)
 |-- frameTagRatio: double (nullable = true)
 |-- hasDomainLink: double (nullable = true)
 |-- html_ratio: double (nullable = true)
 |-- image_ratio: double (nullable = true)
 |-- is_news: double (nullable = true)
 |-- lengthyLinkDomain: double (nullable = true)
 |-- linkwordscore: double (nullable = true)
 |-- news_front_page: double (nullable = true)
 |-- non_markup_alphanum_characters: double (nullable = true)
 |-- numberOfLinks: double (nullable = true)
 |-- numwords_in_url: d

In [18]:
#Step2.5 檢視 DataFrame 'df ', 部份欄位前10筆資料, 注意！'alchemy_category_score','is_news' 原"?" --> 0.0
df.select('url','alchemy_category','alchemy_category_score','is_news','label').show(10)

+--------------------+------------------+----------------------+-------+-----+
|                 url|  alchemy_category|alchemy_category_score|is_news|label|
+--------------------+------------------+----------------------+-------+-----+
|http://www.bloomb...|          business|              0.789131|    1.0|  0.0|
|http://www.popsci...|        recreation|              0.574147|    1.0|  1.0|
|http://www.menshe...|            health|              0.996526|    1.0|  1.0|
|http://www.dumbli...|            health|              0.801248|    1.0|  1.0|
|http://bleacherre...|            sports|              0.719157|    1.0|  0.0|
|http://www.conven...|                 ?|                   0.0|    0.0|  0.0|
|http://gofashionl...|arts_entertainment|               0.22111|    1.0|  1.0|
|http://www.inside...|                 ?|                   0.0|    0.0|  0.0|
|http://www.valetm...|                 ?|                   0.0|    1.0|  1.0|
|http://www.howswe...|                 ?|           

In [19]:
#3.將資料分成train_df 與test_df
#使用DataFrame.randomSplit() 依設定7:3之比例
#並以DataFrame.cache() 暫存於 MM中
#step3.1
train_df,test_df=df.randomSplit([0.7,0.3])
train_df.cache()
test_df.cache()

DataFrame[url: string, alchemy_category: string, alchemy_category_score: double, avglinksize: double, commonlinkratio_1: double, commonlinkratio_2: double, commonlinkratio_3: double, commonlinkratio_4: double, compression_ratio: double, embed_ratio: double, framebased: double, frameTagRatio: double, hasDomainLink: double, html_ratio: double, image_ratio: double, is_news: double, lengthyLinkDomain: double, linkwordscore: double, news_front_page: double, non_markup_alphanum_characters: double, numberOfLinks: double, numwords_in_url: double, parametrizedLinkRatio: double, spelling_errors_ratio: double, label: double]

In [20]:
#4. Spark ML pipeline 流程元件, StringIndexer,OneHotEncoder,VectorAssenbler

In [21]:
#Step4.1, StringIndexer, 匯入StringIndexer module
from pyspark.ml.feature import StringIndexer

In [22]:
#Step4.2 建立 StringIndexer 物件 'categoryIndexer', StringIndexer(inputCol, outputCol)
categoryIndexer=StringIndexer(inputCol='alchemy_category',outputCol='alchemy_category_Index')

In [23]:
#Step4.3 以StringIndexer.fit(DataFrame) method, 產生一個 Transformer 物件 'categoryTransformer'
#              StringIndexer 物件是一個 Estimator 物件
categoryTransformer=categoryIndexer.fit(df)

In [24]:
#Step4.4 查看 Transformer 物件 'categoryTransformer'內容,
#              其中label屬性是一個網頁分類對照表
for i in range(0,len(categoryTransformer.labels)):
    print(str(i)+':'+categoryTransformer.labels[i])

0:?
1:recreation
2:arts_entertainment
3:business
4:health
5:sports
6:culture_politics
7:computer_internet
8:science_technology
9:gaming
10:religion
11:law_crime
12:unknown
13:weather


In [25]:
#Step4.5 使用 'categoryTransformer' 的transform() method 將DataFrame train_df 轉換成 df1
df1=categoryTransformer.transform(train_df)

In [26]:
#Step4.6 檢視轉換後 DataFrame df1 的欄位
df1.columns

['url',
 'alchemy_category',
 'alchemy_category_score',
 'avglinksize',
 'commonlinkratio_1',
 'commonlinkratio_2',
 'commonlinkratio_3',
 'commonlinkratio_4',
 'compression_ratio',
 'embed_ratio',
 'framebased',
 'frameTagRatio',
 'hasDomainLink',
 'html_ratio',
 'image_ratio',
 'is_news',
 'lengthyLinkDomain',
 'linkwordscore',
 'news_front_page',
 'non_markup_alphanum_characters',
 'numberOfLinks',
 'numwords_in_url',
 'parametrizedLinkRatio',
 'spelling_errors_ratio',
 'label',
 'alchemy_category_Index']

In [27]:
#Step4.7 查看轉換的結果, 已將 'alchemy_category'(文字）-->'alchemy_category_Index'(數值）
df1.select("alchemy_category","alchemy_category_Index").show(10)

+------------------+----------------------+
|  alchemy_category|alchemy_category_Index|
+------------------+----------------------+
|        recreation|                   1.0|
|                 ?|                   0.0|
|                 ?|                   0.0|
|                 ?|                   0.0|
|                 ?|                   0.0|
|          business|                   3.0|
|arts_entertainment|                   2.0|
|          business|                   3.0|
|          business|                   3.0|
|                 ?|                   0.0|
+------------------+----------------------+
only showing top 10 rows



In [28]:
#Step5.1 OneHotEncoder: 將一個數字的分類Feature欄位，轉換為多個欄位的Vector. 
#              匯入OneHotEncoder module
from pyspark.ml.feature import OneHotEncoder

In [29]:
#Step5.2  建立 OneHotEncoder 物件 'encoder'
encoder=OneHotEncoder(dropLast=False,
                      inputCol='alchemy_category_Index',
                      outputCol='alchemy_category_IndexVec')

In [30]:
#Step5.3 使用 OneHotEncoder.transform(DataFrame) 將 df1 --> df2
df2=encoder.transform(df1)

In [31]:
df2.columns #新增了 'alchemy_category_IndexVec' 欄位

['url',
 'alchemy_category',
 'alchemy_category_score',
 'avglinksize',
 'commonlinkratio_1',
 'commonlinkratio_2',
 'commonlinkratio_3',
 'commonlinkratio_4',
 'compression_ratio',
 'embed_ratio',
 'framebased',
 'frameTagRatio',
 'hasDomainLink',
 'html_ratio',
 'image_ratio',
 'is_news',
 'lengthyLinkDomain',
 'linkwordscore',
 'news_front_page',
 'non_markup_alphanum_characters',
 'numberOfLinks',
 'numwords_in_url',
 'parametrizedLinkRatio',
 'spelling_errors_ratio',
 'label',
 'alchemy_category_Index',
 'alchemy_category_IndexVec']

In [32]:
#Step5.4  查看轉換後資料
#alchemy_category: 原字串分類, alchemy_category_Index:轉換為一個數字, 
#alchemy_category_IndesVec:轉換為14個欄位的數字 (14,[3],[1.0]) -->這是稀疏向量(SparesVector)表示法
#                                                                         共14個欄位,第3個欄位是1,其餘是0, --> 0,0,0,1,0,0,0,0,0,0,0,0,0,0
df2.select("alchemy_category","alchemy_category_Index","alchemy_category_IndexVec").show(10)

+------------------+----------------------+-------------------------+
|  alchemy_category|alchemy_category_Index|alchemy_category_IndexVec|
+------------------+----------------------+-------------------------+
|        recreation|                   1.0|           (14,[1],[1.0])|
|                 ?|                   0.0|           (14,[0],[1.0])|
|                 ?|                   0.0|           (14,[0],[1.0])|
|                 ?|                   0.0|           (14,[0],[1.0])|
|                 ?|                   0.0|           (14,[0],[1.0])|
|          business|                   3.0|           (14,[3],[1.0])|
|arts_entertainment|                   2.0|           (14,[2],[1.0])|
|          business|                   3.0|           (14,[3],[1.0])|
|          business|                   3.0|           (14,[3],[1.0])|
|                 ?|                   0.0|           (14,[0],[1.0])|
+------------------+----------------------+-------------------------+
only showing top 10 

In [33]:
#Step6.1 VectorAssenbler: 將所有feature欄位整合成1個Vector欄位
#              匯入VectorAssenbler module
from pyspark.ml.feature import VectorAssembler

In [34]:
#Step6.2 建立全部Feature 欄位List 'assemblerInputs'
#'alchemy_category_IndexVec' (14個欄位的0/1)
assemblerInputs=['alchemy_category_IndexVec']+row_df.columns[4:-1]  #[4:-1], 第4到倒數第2
print(assemblerInputs)

['alchemy_category_IndexVec', 'alchemy_category_score', 'avglinksize', 'commonlinkratio_1', 'commonlinkratio_2', 'commonlinkratio_3', 'commonlinkratio_4', 'compression_ratio', 'embed_ratio', 'framebased', 'frameTagRatio', 'hasDomainLink', 'html_ratio', 'image_ratio', 'is_news', 'lengthyLinkDomain', 'linkwordscore', 'news_front_page', 'non_markup_alphanum_characters', 'numberOfLinks', 'numwords_in_url', 'parametrizedLinkRatio', 'spelling_errors_ratio']


In [35]:
#Step6.3 建立VectorAssenbler 物件 'assembler'
assembler=VectorAssembler(inputCols=assemblerInputs,outputCol="features")

In [36]:
#Step6.4 執行VectorAssenbler.transform(DataFrame) 轉換
df3=assembler.transform(df2)

In [37]:
#Step6.5 檢視整合後結果, 新增了 'features' 欄位
df3.columns

['url',
 'alchemy_category',
 'alchemy_category_score',
 'avglinksize',
 'commonlinkratio_1',
 'commonlinkratio_2',
 'commonlinkratio_3',
 'commonlinkratio_4',
 'compression_ratio',
 'embed_ratio',
 'framebased',
 'frameTagRatio',
 'hasDomainLink',
 'html_ratio',
 'image_ratio',
 'is_news',
 'lengthyLinkDomain',
 'linkwordscore',
 'news_front_page',
 'non_markup_alphanum_characters',
 'numberOfLinks',
 'numwords_in_url',
 'parametrizedLinkRatio',
 'spelling_errors_ratio',
 'label',
 'alchemy_category_Index',
 'alchemy_category_IndexVec',
 'features']

In [38]:
#Step6.6 查看 features 欄位內容
df3.select("features").take(2)

[Row(features=SparseVector(36, {1: 1.0, 14: 0.3034, 15: 2.3529, 16: 0.7228, 17: 0.375, 18: 0.3043, 19: 0.288, 20: 0.483, 23: 0.034, 25: 0.223, 26: 0.2186, 27: 1.0, 28: 1.0, 29: 14.0, 31: 9935.0, 32: 184.0, 33: 3.0, 34: 0.3478, 35: 0.1383})),
 Row(features=SparseVector(36, {0: 1.0, 15: 0.3529, 20: 0.5749, 23: 0.0319, 25: 0.3047, 26: 0.128, 29: 6.0, 31: 446.0, 32: 17.0, 34: 0.0588, 35: 0.08}))]

In [39]:
#Step7: 使用DecisionTreeClassifier 二元分類
#            step1: 使用 fit() method, 進行訓練產生DecisionTree模型
#            step2: 使用transform() 進行轉換,轉換後會產生預測結果

In [40]:
#Step7.1 匯入 DecisionTreeClassifier module
from pyspark.ml.classification import DecisionTreeClassifier

In [41]:
#Step7.2 執行 DecisionTreeClassifier() 建立 DecisionTreeClassifier 物件 dt
dt=DecisionTreeClassifier(labelCol="label",  #label欄位
                          featuresCol="features",        #feature 欄位
                          impurity="gini",                     #分裂原則--> 吉尼值最小者
                          maxDepth=10,                       #最大深度
                          maxBins=14)                          #最大分裂數

In [42]:
type(dt)

pyspark.ml.classification.DecisionTreeClassifier

In [43]:
#Step7.3 執行訓練, 使用 DesisionTreeClassifier.fit() method
dt_model=dt.fit(df3)

In [44]:
print(dt_model)

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_444da4ea5f751e8d2aa5) of depth 10 with 643 nodes


In [45]:
#Step7.4 進行預測, 使用df_model.transform(DataFrame)
df4=dt_model.transform(df3)

In [46]:
df4.printSchema()

root
 |-- url: string (nullable = true)
 |-- alchemy_category: string (nullable = true)
 |-- alchemy_category_score: double (nullable = true)
 |-- avglinksize: double (nullable = true)
 |-- commonlinkratio_1: double (nullable = true)
 |-- commonlinkratio_2: double (nullable = true)
 |-- commonlinkratio_3: double (nullable = true)
 |-- commonlinkratio_4: double (nullable = true)
 |-- compression_ratio: double (nullable = true)
 |-- embed_ratio: double (nullable = true)
 |-- framebased: double (nullable = true)
 |-- frameTagRatio: double (nullable = true)
 |-- hasDomainLink: double (nullable = true)
 |-- html_ratio: double (nullable = true)
 |-- image_ratio: double (nullable = true)
 |-- is_news: double (nullable = true)
 |-- lengthyLinkDomain: double (nullable = true)
 |-- linkwordscore: double (nullable = true)
 |-- news_front_page: double (nullable = true)
 |-- non_markup_alphanum_characters: double (nullable = true)
 |-- numberOfLinks: double (nullable = true)
 |-- numwords_in_url: d

In [47]:
df4.select("url","label").show(10)

+--------------------+-----+
|                 url|label|
+--------------------+-----+
|http://1000awesom...|  1.0|
|http://100miledie...|  0.0|
|http://123healtht...|  1.0|
|http://13gb.com/p...|  0.0|
|http://13gb.com/v...|  0.0|
|http://17andbakin...|  1.0|
|http://17andbakin...|  1.0|
|http://17andbakin...|  1.0|
|http://1greengene...|  1.0|
|http://1x.com/pho...|  1.0|
+--------------------+-----+
only showing top 10 rows



In [48]:
#8. 將上述Machine Learning 流程各元件：StringIndexer, OneHotEncoder, VectorAssembler, DecisionTreeClassifier
#    構成一個pipeline 物件, Machine Learning Pipeline 物件

In [49]:
#Step8.1 匯入全部需要的模組
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer,OneHotEncoder,VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier

In [52]:
#Step8.2 建立 Pipeline 物件 'pipeline'
categoryIndexer=StringIndexer(inputCol='alchemy_category',       #StringIndexer 物件 'categoryIndexer'
                              outputCol='alchemy_category_Index')
encoder=OneHotEncoder(dropLast=False,                                        #OneHotEncoder 物件 'encoder'
                      inputCol='alchemy_category_Index',
                      outputCol='alchemy_category_IndexVec')
assemblerInputs=['alchemy_category_IndexVec']+row_df.columns[4:-1]  #全部Feature 欄位List 物件 'assemblerInputs'
assembler=VectorAssembler(inputCols=assemblerInputs,outputCol="features")  #VectorAssenbler 物件 'assembler'
dt=DecisionTreeClassifier(labelCol="label",         #lDecisionTreeClassifier 物件 dt
                          featuresCol="features",       
                          impurity="gini",                    
                          maxDepth=10,                       
                          maxBins=14)    
#---------------------------------------------------------------------------------------------------------------------------------------------------
pipeline=Pipeline(stages=[categoryIndexer,encoder,assembler,dt])    #Pipeline 物件 'pipeline'

In [53]:
#Step8.3 查看 Pipeline stages, getStages() 
pipeline.getStages()

[StringIndexer_4c92be9160ecd3acf197,
 OneHotEncoder_47dea4860e8c6dd6dfe7,
 VectorAssembler_41a2b09a2e5b1c4097aa,
 DecisionTreeClassifier_42f7a41ee6f8a0f1364d]

In [54]:
#9. 使用 ML pipeline 進行資料處理與訓練
#Step9.1 使用 Pipeline.fit() 進行訓練, 傳入 train_df 訓練資料
pipelineModel=pipeline.fit(train_df)

In [55]:
type(pipelineModel)

pyspark.ml.pipeline.PipelineModel

In [56]:
#Step9.2 檢視訓練後的決策樹模型
pipelineModel.stages[3]

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_42f7a41ee6f8a0f1364d) of depth 10 with 643 nodes

In [57]:
#Step9.3 檢視訓練後的決策樹模型規則, toDebugString
print(pipelineModel.stages[3].toDebugString)

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_42f7a41ee6f8a0f1364d) of depth 10 with 643 nodes
  If (feature 31 <= 1803.0)
   If (feature 2 in {1.0})
    If (feature 35 <= 0.166666667)
     If (feature 31 <= 1337.0)
      If (feature 26 <= 0.199304751)
       If (feature 25 <= 0.167384934)
        Predict: 1.0
       Else (feature 25 > 0.167384934)
        If (feature 14 <= 0.736574)
         If (feature 33 <= 11.0)
          If (feature 27 <= 0.0)
           Predict: 0.0
          Else (feature 27 > 0.0)
           If (feature 28 <= 0.0)
            Predict: 0.0
           Else (feature 28 > 0.0)
            Predict: 0.0
         Else (feature 33 > 11.0)
          Predict: 1.0
        Else (feature 14 > 0.736574)
         If (feature 20 <= 0.461231995)
          Predict: 1.0
         Else (feature 20 > 0.461231995)
          If (feature 16 <= 0.239766082)
           If (feature 15 <= 1.03125)
            Predict: 0.0
           Else (feature 15 > 1.03125)
            Pre

In [58]:
#10. 使用pipelineModel 進行預測
# Step10.1 使用pilelineModel.transform(), 傳入 test_df 測試資料進行預測
predicted=pipelineModel.transform(test_df)

In [59]:
type(predicted)

pyspark.sql.dataframe.DataFrame

In [60]:
#Step10.2 檢視預測結果 DataFrame 'predicted' 欄位,
#新增了rawPrediction', 'probability', 'prediction' 3個欄位
predicted.columns   

['url',
 'alchemy_category',
 'alchemy_category_score',
 'avglinksize',
 'commonlinkratio_1',
 'commonlinkratio_2',
 'commonlinkratio_3',
 'commonlinkratio_4',
 'compression_ratio',
 'embed_ratio',
 'framebased',
 'frameTagRatio',
 'hasDomainLink',
 'html_ratio',
 'image_ratio',
 'is_news',
 'lengthyLinkDomain',
 'linkwordscore',
 'news_front_page',
 'non_markup_alphanum_characters',
 'numberOfLinks',
 'numwords_in_url',
 'parametrizedLinkRatio',
 'spelling_errors_ratio',
 'label',
 'alchemy_category_Index',
 'alchemy_category_IndexVec',
 'features',
 'rawPrediction',
 'probability',
 'prediction']

In [61]:
#Step10.3 查看預測結果, 'prediction': 預測結果 0 或 1
predicted.select('url','features','rawPrediction', 'probability','label', 'prediction').show(10)

+--------------------+--------------------+-------------+--------------------+-----+----------+
|                 url|            features|rawPrediction|         probability|label|prediction|
+--------------------+--------------------+-------------+--------------------+-----+----------+
|http://1000awesom...|(36,[0,15,16,17,1...|  [20.0,74.0]|[0.21276595744680...|  1.0|       1.0|
|http://1000awesom...|(36,[2,14,15,16,1...|  [26.0,84.0]|[0.23636363636363...|  1.0|       1.0|
|http://2oddities....|(36,[7,14,15,16,1...| [161.0,62.0]|[0.72197309417040...|  1.0|       0.0|
|http://30aeats.co...|(36,[0,15,16,17,1...|   [3.0,20.0]|[0.13043478260869...|  1.0|       1.0|
|http://5secondrul...|(36,[10,14,15,16,...|  [53.0,20.0]|[0.72602739726027...|  1.0|       0.0|
|http://98smile.co...|(36,[5,14,15,16,2...|   [17.0,4.0]|[0.80952380952380...|  0.0|       0.0|
|http://98smile.co...|(36,[4,14,15,16,2...|    [7.0,0.0]|           [1.0,0.0]|  1.0|       0.0|
|http://abeautiful...|(36,[2,14,15,16,1.

In [62]:
#Step10.4 查看預測結果與機率 'probability',
#                probability 值是 DenseVector 格式是 [預測是0的機率, 預測是1的機率]
predicted.select('probability','prediction').take(10)

[Row(probability=DenseVector([0.2128, 0.7872]), prediction=1.0),
 Row(probability=DenseVector([0.2364, 0.7636]), prediction=1.0),
 Row(probability=DenseVector([0.722, 0.278]), prediction=0.0),
 Row(probability=DenseVector([0.1304, 0.8696]), prediction=1.0),
 Row(probability=DenseVector([0.726, 0.274]), prediction=0.0),
 Row(probability=DenseVector([0.8095, 0.1905]), prediction=0.0),
 Row(probability=DenseVector([1.0, 0.0]), prediction=0.0),
 Row(probability=DenseVector([0.5556, 0.4444]), prediction=0.0),
 Row(probability=DenseVector([0.1022, 0.8978]), prediction=1.0),
 Row(probability=DenseVector([0.6471, 0.3529]), prediction=0.0)]

In [63]:
#11. 評估模型的準確率
#      我們已經使用建立的模型作預測, 現在我們將以 BinaryClassificationEvaluator module 作評估

In [64]:
#Step11.1 匯入模組
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [65]:
#Step11.2 建立 BinaryClassificationEvaluator 物件 evaluator
evaluator=BinaryClassificationEvaluator(rawPredictionCol="rawPrediction",  #之前預測後產生的欄位
                                        labelCol="label",                                                              #label欄位
                                        metricName="areaUnderROC")                                      #area under ROC --> AUC

In [66]:
#Step11.3 計算AUC
predictions=pipelineModel.transform(test_df)  #predictions 預測值
auc=evaluator.evaluate(predictions)                   #以BinaryClassifictionEvaluator.evaluate() 做評估, 計算 AUC
auc                                                                          #display auc

0.6550426589264904

In [67]:
#12.找出最佳模型
#使用Spark  ML Pipeline訓練驗證(TrainValidation)與交叉驗證(crossValidation)找出最佳模型

In [70]:
#Step12.1 匯入模組 ParamGridBuilder,TrainValidationSplit
from pyspark.ml.tuning import ParamGridBuilder,TrainValidationSplit

In [71]:
#Step12.2 設定訓練驗證的參數, -->Step7.2 執行 DecisionTreeClassifier() 的參數：
#                                                                               impurity(分裂原則),maxDepth(最大深度),maxBins(最大分裂數)
paramGrid=ParamGridBuilder().addGrid(dt.impurity,["gini","entropy"])\    #impurity 用 ["gini","entropy"] 2類
.addGrid(dt.maxDepth,[5,10,15])\                                                                     #maxDepth 用  [5,10,15]           3種
.addGrid(dt.maxBins,[10,15,20])\                                                                      #maxBins 用 [10,15,20]             3種  
.build()                                                                                                                  #共執行 2x3x3=18次

In [73]:
#Step12.3 建立TrainValidationSplit 物件 tvs
tvs=TrainValidationSplit(estimator=df,                                  #DecisionTreeClassifier 'df'
                         evaluator=evaluator,                                      #BinaryClassificationEvaluator 'evalutor'
                         estimatorParamMaps=paramGrid,               #ParamGridBuilder 'paramGrid'
                         trainRatio=0.8)                                                #訓練驗證前,會先將資料依8:2分成訓練資料與驗證資料

In [77]:
#Step12.4 建立 tvs_pipeline, 與之前 pipeline相同, 除最後加上一個 TrainValidationSplit 物件 tvs
tvs_pipeline=Pipeline(stages=[categoryIndexer,encoder,assembler,tvs])

In [78]:
#Step12.5  使用tvs_pipeline 進行 "訓練驗證",以tvs_pipeline.fit(train_df)
tvs_pipelineModel=tvs_pipeline.fit(train_df)

In [79]:
#Step12.6 檢視訓練完成的 "最佳模型",  使用 Model.stage.bestModel 屬性
tvs_pipeline.getStages() #TrainValidationSplit 在stages[3]  (第四個 stage)

[StringIndexer_4c92be9160ecd3acf197,
 OneHotEncoder_47dea4860e8c6dd6dfe7,
 VectorAssembler_41a2b09a2e5b1c4097aa,
 TrainValidationSplit_4578b573f30da71c2fb4]

In [81]:
bestModel=tvs_pipelineModel.stages[3].bestModel
bestModel

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_4d25a02b86a56789c555) of depth 10 with 643 nodes

In [83]:
#Step12.7 檢視訓練驗證完成的最佳模型規則
print(bestModel.toDebugString[:500])  #只顯示前500個

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_4d25a02b86a56789c555) of depth 10 with 643 nodes
  If (feature 31 <= 1803.0)
   If (feature 2 in {1.0})
    If (feature 35 <= 0.166666667)
     If (feature 31 <= 1337.0)
      If (feature 26 <= 0.199304751)
       If (feature 25 <= 0.167384934)
        Predict: 1.0
       Else (feature 25 > 0.167384934)
        If (feature 14 <= 0.736574)
         If (feature 33 <= 11.0)
          If (feature 27 <= 0.0)
           Predict: 0.0
         


In [84]:
#Step12.8 評估最佳模型 AUC
predictions=tvs_pipelineModel.transform(test_df)
auc=evaluator.evaluate(predictions)
auc

0.6550426589264904

In [85]:
#Step13 使用CrossValidaton交叉驗證找出最佳模型
#            交叉驗證:(以k-Fold, k=3為例)

In [86]:
#Step13.1 匯入模組
from pyspark.ml.tuning import CrossValidator

In [87]:
#Step13.2 建立CrossValidator 物件 cv
cv=CrossValidator(estimator=dt,                              #DecisionTreeClassifier 'df'
                  evaluator=evaluator,                               #BinaryClassificationEvaluator 'evalutor'
                  estimatorParamMaps=paramGrid,        #ParamGridBuilder 'paramGrid'(impurity:2, maxDepth:3, maxBins:3)
                  numFolds=3)                                             #k=3,   2*3*3*3=54 次

In [88]:
#Step13.3 建立交叉驗證 pipeline, 'cv_pipeline'
cv_pipeline=Pipeline(stages=[categoryIndexer,encoder,assembler,cv])

In [89]:
#Step13.4 使用cv_pipeline.fit(DataFrame) 進行交叉驗證, 產生model cv_pipelineModel
cv_pipelineModel=cv_pipeline.fit(train_df)

In [90]:
#Step13.5 檢視交叉驗證完成的最佳模型
cv_pipeline.getStages()    #CrossValidator 在stages[3], 第4個stage

[StringIndexer_4c92be9160ecd3acf197,
 OneHotEncoder_47dea4860e8c6dd6dfe7,
 VectorAssembler_41a2b09a2e5b1c4097aa,
 CrossValidator_49198d37d9ed4762d1a9]

In [91]:
bestModel=cv_pipelineModel.stages[3].bestModel
bestModel

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_42f7a41ee6f8a0f1364d) of depth 15 with 1675 nodes

In [92]:
#Step13.6 檢視訓練驗證完成的最佳模型規則
print(bestModel.toDebugString[:500])  #只顯示前500個

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_42f7a41ee6f8a0f1364d) of depth 15 with 1675 nodes
  If (feature 31 <= 1236.0)
   If (feature 2 in {1.0})
    If (feature 35 <= 0.171201814)
     If (feature 25 <= 0.267837001)
      If (feature 26 <= 0.09352518)
       If (feature 25 <= 0.166314085)
        Predict: 1.0
       Else (feature 25 > 0.166314085)
        If (feature 33 <= 0.0)
         Predict: 1.0
        Else (feature 33 > 0.0)
         If (feature 29 <= 30.0)
          Pr


In [93]:
#Step13.7 評估最佳模型 AUC
predictions=cv_pipelineModel.transform(test_df)
auc=evaluator.evaluate(predictions)
auc

0.6693143944034504

In [94]:
#Step14. Spark 機器學習RandomForest隨機森林分類器

In [96]:
#Step14.1 建立RandomForestClassifier Pipeline
from pyspark.ml.classification import RandomForestClassifier   #匯入 RandomForestClassifier

rf=RandomForestClassifier(labelCol='label',                  #建立 RandomForestClassifier 物件 'rf'              
                          featuresCol='features',
                          numTrees=10)

rfpipeline=Pipeline(stages=[categoryIndexer,encoder,assembler,rf]) # 建立 Pipeline 物件 'rfpipeline'

In [97]:
#Step14.2 評估 RandomForestClassifier 的準確
rfpipelineModel=rfpipeline.fit(train_df)  #以rfpipeline.fit(), 傳入train_df 進行訓練,產生模型 'rfpipelineModel'
rfpredicted=rfpipelineModel.transform(test_df)  #以模型進行預測
auc=evaluator.evaluate(rfpredicted) #計算AUC, AUC 比之前模型進步了！
auc

0.7340519854620389

In [102]:
#Step14.3 使用RandomForestClassifier TrainValidation (訓練驗證) 找出最佳模型
# import 所需模組, ParamGridBuilder, TrainValidationSplit, BinaryClassificationEvaluator, RandomForestClassifier,
from pyspark.ml.tuning import ParamGridBuilder,TrainValidationSplit
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.classification import RandomForestClassifier

# 建立 ParamGridBuilder 物件 paramGrid
paramGrid=ParamGridBuilder().addGrid(rf.impurity,["gini","entropy"])\
.addGrid(rf.maxDepth,[5,10,15])\
.addGrid(rf.maxBins,[10,20,30])\
.build()

# 建立 TrainValidationSplit 訓練驗證 物件 'rftvs' ,訓練參數 -->paramGrid
rftvs=TrainValidationSplit(estimator=rf,
                          evaluator=evaluator,
                          estimatorParamMaps=paramGrid,
                          trainRatio=0.8)

# 建立 Pipeline 物件 'rftvs_pipeline', 以Pipeline.fit() 訓練資料 train_df 得到model 'rftvs_pipelineModel,
#以模型 rftvs_pipelineModel 對測試資料 test.df 作預測, 結果為 rftvspredictions
rftvs_pipeline=Pipeline(stages=[categoryIndexer,encoder,assembler,rftvs])  #Pipeline 物件rftvs_pipeline
rftvs_pipelineModel=rftvs_pipeline.fit(train_df)        #以rftvs_pipeline.fit() --> Model 'rftvs_pipelineModel'
rftvspredictions=rftvs_pipelineModel.transform(test_df)  #以模型 rftvs_pipelineModel 對測試資料 test.df 作預測, 結果為 rftvspredictions
auc=evaluator.evaluate(rftvspredictions) #以evaluator 評估rftvspredictions (計算AUC)
auc

0.724546395509189

In [105]:
#Step14.4 使用RandomForestClassifier  作 crossValidation 交叉驗證 (k=3) 找出最佳模型
# import 所需模組, ParamGridBuilder, TrainValidationSplit, BinaryClassificationEvaluator, RandomForestClassifier,
#                               CrossValidator
#from ......         #pyspark.ml.tuning   -->ParamGridBuilder,TrainValidationSplit
#from ......         #pyspark.ml.evaluation  -->BinaryClassificationEvaluator
#from ......         #pyspark.ml.classification -->RandomForestClassifier
#from ......         #pyspark.ml.tuning -->CrossValidator

# 建立 ParamGridBuilder 物件 paramGrid
#paramGrid=

# 建立 CrossValidator 交叉訓練驗證 物件 'rfcv' ,訓練參數 -->paramGrid, k=3
#rfcv=CrossValidator(......
                         

# 建立 Pipeline 物件 'rfcv_pipeline', 以Pipeline.fit() 訓練資料 train_df 得到model 'rfcv_pipelineModel,
#以模型 rfcv_pipelineModel 對測試資料 test.df 作預測, 結果為 rfcvpredictions
#......

0.7438775958961981

In [110]:
#Step14.5 使用最佳模型預測結果
DescDict={                                           #Dictionary DescDict
    0:"(ephemeral)",
    1:"(evergreen)"}

for data in rfcvpredictions.select('url','prediction').take(5):
    print("URL Address:"+str(data[0])+"\n"+ \
                             "   預測是:"+str(data[-1])+ \
                             "   說明: "+DescDict[data[-1]]+"\n")

URL Address:http://1000awesomethings.com/2008/07/07/989-blowing-your-nose-in-the-shower/
   預測是:1.0   說明: (evergreen)

URL Address:http://1000awesomethings.com/2008/12/29/864-mastering-the-art-of-the-all-you-can-eat-buffet/
   預測是:1.0   說明: (evergreen)

URL Address:http://2oddities.com/two-friends-talking-after-the-summer-holidays/
   預測是:0.0   說明: (ephemeral)

URL Address:http://30aeats.com/recipes/sun-dried-tomato-kalamata-olive-and-basil-pesto-focaccia/
   預測是:1.0   說明: (evergreen)

URL Address:http://5secondrule.typepad.com/my_weblog/2010/02/recipe-for-easy-potsticker-gyoza-dumplings.html
   預測是:0.0   說明: (ephemeral)



In [106]:
rfcvpredictions.printSchema()

root
 |-- url: string (nullable = true)
 |-- alchemy_category: string (nullable = true)
 |-- alchemy_category_score: double (nullable = true)
 |-- avglinksize: double (nullable = true)
 |-- commonlinkratio_1: double (nullable = true)
 |-- commonlinkratio_2: double (nullable = true)
 |-- commonlinkratio_3: double (nullable = true)
 |-- commonlinkratio_4: double (nullable = true)
 |-- compression_ratio: double (nullable = true)
 |-- embed_ratio: double (nullable = true)
 |-- framebased: double (nullable = true)
 |-- frameTagRatio: double (nullable = true)
 |-- hasDomainLink: double (nullable = true)
 |-- html_ratio: double (nullable = true)
 |-- image_ratio: double (nullable = true)
 |-- is_news: double (nullable = true)
 |-- lengthyLinkDomain: double (nullable = true)
 |-- linkwordscore: double (nullable = true)
 |-- news_front_page: double (nullable = true)
 |-- non_markup_alphanum_characters: double (nullable = true)
 |-- numberOfLinks: double (nullable = true)
 |-- numwords_in_url: d

In [108]:
rfcvpredictions.take(5)

[Row(url=u'http://1000awesomethings.com/2008/07/07/989-blowing-your-nose-in-the-shower/', alchemy_category=u'?', alchemy_category_score=0.0, avglinksize=2.144628099, commonlinkratio_1=0.796875, commonlinkratio_2=0.39453125, commonlinkratio_3=0.33203125, commonlinkratio_4=0.3203125, compression_ratio=0.502245815, embed_ratio=0.0, framebased=0.0, frameTagRatio=0.027994792, hasDomainLink=0.0, html_ratio=0.189758995, image_ratio=0.235416667, is_news=1.0, lengthyLinkDomain=1.0, linkwordscore=17.0, news_front_page=0.0, non_markup_alphanum_characters=10588.0, numberOfLinks=256.0, numwords_in_url=5.0, parametrizedLinkRatio=0.3828125, spelling_errors_ratio=0.136752137, label=1.0, alchemy_category_Index=0.0, alchemy_category_IndexVec=SparseVector(14, {0: 1.0}), features=SparseVector(36, {0: 1.0, 15: 2.1446, 16: 0.7969, 17: 0.3945, 18: 0.332, 19: 0.3203, 20: 0.5022, 23: 0.028, 25: 0.1898, 26: 0.2354, 27: 1.0, 28: 1.0, 29: 17.0, 31: 10588.0, 32: 256.0, 33: 5.0, 34: 0.3828, 35: 0.1368}), rawPredict